In [ ]:
import openai


In [1]:
with open("raw_Metadata_Image8_full.txt") as f:
    metadata = f.read()

In [3]:
messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Please transcribe the following metadata such that it follows the latest ome xml schema."},
    {"role": "assistant", "content": "Sure, please provide the raw metadata such that I can transcribe it."},
    {"role": "user", "content": metadata}
]

In [8]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k",
    messages=messages,
    temperature=0,
    max_tokens=5000,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    stop=["\"\"\""]
)
print(response["choices"][0]["message"]["content"])

Here is the transcribed metadata following the latest OME XML schema:

```xml
<?xml version="1.0" encoding="UTF-8"?>
<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
  <Image ID="Image_8">
    <Pixels BigEndian="false" DimensionOrder="XYCZT" Interleaved="false" PixelType="uint8" SizeC="3" SizeT="30" SizeX="680" SizeY="280" SizeZ="1">
      <Channel ID="Channel_1" SamplesPerPixel="1">
        <LightPath>
          <IlluminationChannel ID="IlluminationChannel_1">
            <LightSourceRef ID="LightSource_1"/>
            <DetectorRef ID="Detector_1"/>
          </IlluminationChannel>
        </LightPath>
      </Channel>
      <Channel ID="Channel_2" SamplesPerPixel="1">
        <LightPath>
          <IlluminationChannel ID="IlluminationChannel_2">
            <LightSourceRef ID="LightSource

In [4]:
import instructor
from openai import OpenAI
from pydantic import BaseModel

client = instructor.patch(OpenAI())

class UserDetail(BaseModel):
    name: str
    age: int
    
user = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=UserDetail,
    messages=[
        {"role": "user", "content": "Extract Aaron is 25 years old"},
    ],
)
assert isinstance(user, UserDetail)
assert user.name == "Aaron"
assert user.age == 25
print(user.model_dump_json(indent=2))

{
  "name": "Aaron",
  "age": 25
}


In [5]:
print(user._raw_response.model_dump_json(indent=2))

{
  "id": "chatcmpl-9HU1jwSUryakJkn3BflKfRzhwhfBZ",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": [
          {
            "id": "call_w4tRlHdD10JWUqfrBnLGasxT",
            "function": {
              "arguments": "{\"name\":\"Aaron\",\"age\":25}",
              "name": "UserDetail"
            },
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1713954027,
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "system_fingerprint": "fp_c2295e73ad",
  "usage": {
    "completion_tokens": 9,
    "prompt_tokens": 81,
    "total_tokens": 90
  }
}


In [18]:
from openai import OpenAI
from pydantic import BaseModel
import instructor

# Apply the patch to the OpenAI client
# enables response_model keyword
client = instructor.patch(OpenAI())

class QuestionAnswer(BaseModel):
    question: str
    answer: str

question = "What is the meaning of life?"
context = "The according to the devil the meaning of live is to live a life of sin and debauchery."

qa: QuestionAnswer = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=QuestionAnswer,
    messages=[
        {
            "role": "system",
            "content": "You are a system that answers questions based on the context. answer exactly what the question asks using the context.",
        },
        {
            "role": "user",
            "content": f"using the context: {context}\n\nAnswer the following question: {question}",
        },
    ],
)
print(qa.model_dump_json(indent=2))

{
  "question": "What is the meaning of life?",
  "answer": "According to the devil, the meaning of life is to live a life of sin and debauchery."
}


# LLM_Validator validation doesnt seem to work so well for me :(

In [1]:
from pydantic import BaseModel, BeforeValidator
from typing_extensions import Annotated
from instructor import llm_validator

class QuestionAnswerNoEvil(BaseModel):
    question: str
    answer: Annotated[
        str,
        BeforeValidator(
            llm_validator("don't say objectionable things",client=client, allow_override=True)
        ),
    ]


try:
    qa: QuestionAnswerNoEvil = client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_model=QuestionAnswerNoEvil,
        messages=[
            {
                "role": "system",
                "content": "You are a system that answers questions based on the context. answer exactly what the question asks using the context.",
            },
            {
                "role": "user",
                "content": f"using the context: {context}\n\nAnswer the following question: {question}",
            },
        ],
    )
except Exception as e:
    print(e)
    
print(qa.model_dump_json(indent=2))

NameError: name 'client' is not defined

In [3]:
import datetime
from openai import OpenAI
from typing import List
from pydantic import Field
from typing import Iterable, Optional
import instructor

client = instructor.patch(OpenAI(), mode=instructor.Mode.FUNCTIONS)


In [4]:
class Property(BaseModel):
    index: str = Field(..., description="Monotonically increasing ID")
    key: str = Field(description="Must be snake case")
    value: str

class Person(BaseModel):
    name: str
    age: int
    birthday: datetime.date
    properties: List[Property] = Field(
        ...,
        description="Numbered list of arbitrary extracted properties, should be exactly 2"
    )
    
class MaybePerson(BaseModel):
    result: Optional[Person] = Field(default=None)
    error: bool = Field(default=False)
    message: Optional[str]
    
    
schema = MaybePerson.model_json_schema()


def extract(content):
    return client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user",
             "content": f"Extract '{content}'"}],
        response_model=MaybePerson
    )

In [6]:
content = f"Aaron is 25 years old his birthday was yesterday today is {datetime.datetime}. He is friends with Caro whichs brithday was last week. They both like json."
extract(content).model_dump()

{'result': {'name': 'Aaron',
  'age': 25,
  'birthday': datetime.date(1997, 3, 20),
  'properties': [{'index': '1', 'key': 'friends', 'value': 'Caro'},
   {'index': '2', 'key': 'likes_json', 'value': 'true'}]},
 'error': False,
 'message': 'Person extracted successfully.'}

In [23]:
content = (f'Today is {datetime.datetime}, yesterday Aaron did a lighsheet microscopy experiment with the following metadata. <Image ID="Image:0" Name="Image 8 #1">'
           '<Pixels BigEndian="false" DimensionOrder="XYCZT"'
           'ID="Pixels:0"'
           'PhysicalSizeX="0.0992287815904495"'
           'PhysicalSizeXUnit="µm"'
           'PhysicalSizeY="0.0992287815904495"'
           'PhysicalSizeYUnit="µm"'
           'SignificantBits="8"'
           'SizeC="3"'
           'SizeT="30"'
           'SizeX="680"'
           'SizeY="280"'
           'SizeZ="1"'
           'Type="uint8">'
           '<Channel AcquisitionMode="LaserScanningConfocalMicroscopy" Color="-16711681" EmissionWavelength="646.4456100000001" EmissionWavelengthUnit="nm" ExcitationWavelength="561.0" ExcitationWavelengthUnit="nm" Fluor="mRFP1.2" ID="Channel:0:0" IlluminationType="Epifluorescence" Name="ChS2-T1" SamplesPerPixel="1">'
           '<DetectorSettings Binning="1x1" ID="Detector:0:0"/>'
           '<LightPath/>'
           '</Channel>'
           '<Channel AcquisitionMode="LaserScanningConfocalMicroscopy" Color="-1" ID="Channel:0:1" IlluminationType="Epifluorescence" Name="T PMT-T1" SamplesPerPixel="1">'
           '<DetectorSettings Binning="1x1" ID="Detector:0:1"/>'
           '<LightPath/>'
           '</Channel>'
           '<Channel AcquisitionMode="LaserScanningConfocalMicroscopy" Color="16711935" EmissionWavelength="530.4922920000002" EmissionWavelengthUnit="nm" ExcitationWavelength="488.00000000000006" ExcitationWavelengthUnit="nm" Fluor="EGFP" ID="Channel:0:2" IlluminationType="Epifluorescence" Name="ChS1-T2" SamplesPerPixel="1">'
           '<DetectorSettings Binning="1x1" ID="Detector:1:0"/>'
           '<LightPath/>'
           '</Channel>'
           '<TiffData FirstC="0" FirstT="0" FirstZ="0" IFD="0" PlaneCount="1">'
           '<UUID FileName="testetst_Image8_edited_.ome.tif">urn:uuid:27555393-9fb6-4c14-942c-badbf7548154</UUID>'
           '</TiffData>'
           '</Pixels>'
           '</Image>')

In [49]:
from ome_types import OME
type(OME)

pydantic._internal._model_construction.ModelMetaclass

In [41]:
from ome_types.model import Image
from ome_types.model import Experiment
from ome_types.model import Experimenter
from ome_types.model import Annotation
    
    
schema = OME.model_json_schema()
print(schema)

TypeError: unhashable type: 'dict'

In [46]:
def extract(content):
    return client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user",
             "content": f"Extract the ome image'{content}'"}],
        response_model=OME
    )

In [47]:
extract(content).model_dump()

TypeError: unhashable type: 'dict'

In [34]:
import ome_types
from ome_types._autogenerated import ome_2016_06 as OME
from ome_types._mixins._base_type import OMEType
from pydantic import BaseModel
import instructor
from openai import OpenAI

client = instructor.patch(OpenAI(), mode=instructor.Mode.FUNCTIONS)



def extract(content):
    return client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user",
             "content": f"Extract '{content}'"}],
        response_model=OME.Image
    )

content= (" BitsPerPixel	8"
          "DimensionOrder	XYCZT"
          "IsInterleaved	false"
          "IsRGB	false"
          "LittleEndian	true"
          "PixelType	uint8"
          "Series 0 Name	Image 8 #1"
          "SizeC	3"
          "SizeT	30"
          "SizeX	680"
          "SizeY	280"
          "SizeZ	1")

extract(content).model_dump()

BadRequestError: Error code: 400 - {'error': {'message': '"This element describes the actual image and its meta-data.\\n\\nThe elements that are references (ending in Ref or Settings) refer\\nto elements defined outside of the Image element. Ref elements are\\nsimple links, while Settings elements are links with additional\\nvalues. If any of the required Image attributes or elements are\\nmissing, its guaranteed to be an invalid document. The required\\nattributes and elements are ID and Pixels. ExperimenterRef is\\nrequired for all Images with well formed LSIDs. ImageType is a\\nvendor-specific designation of the type of image this is. Examples\\nof ImageType include \'STK\', \'SoftWorx\', etc. The Name attributes are\\nin all cases the name of the element instance. In this case, the\\nname of the image, not necessarily the filename. Physical size of\\npixels are microns[µm].\\n\\nAttributes\\n----------\\nacquisition_date : None | datetime\\n    The acquisition date of the Image. Th